In [ ]:
import numpy as np
import random as rn
import seaborn as sns
import pandas as pd
import matplotlib
import math
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
data = pd.read_csv("heart.csv")

In [ ]:
#Wyrzucamy duplikaty
data = data.drop_duplicates(ignore_index=True)

In [ ]:
#Sprawdzamy czy są jakieś wartości NULL
data.isna().any().any()

#Czy da się jeszcze w jakiś sposób przygotować dane?
#Sprawdziłem, że nie ma stringów tam gdzie powinny być inty, nie ma pustych komórek
#Sprawdziłem, czy nie ma dziwnych liczb, np w płci nagle 364, albo 1400 cholesterolu
#Nie mam wiedzy, żeby zweryfiukować czy zakres tych danych jest możliwy tj. czy taki niski/wysoki cholesterol może się pojawić
#Niżej wybieramy najbardziej skorelowane kolumny i na nich bazujemy - bo płeć ma się nijak do wyniku
#Robię to tak, a nie algorytmem PCA - w zasadzie nie wiem czemu, może to będzie w stanie jakoś poprawić wyniki

In [ ]:
target_col = "output"
corr_matrix = data.corr()[target_col]
top_cols = corr_matrix.abs().sort_values(ascending=False)
top_cols.plot(kind="bar")

In [ ]:
class DataProcessing:
    @staticmethod
    def shuffle(x):
        for i in range(len(x)-1, 0, -1):
            j = rn.randint(0, i-1)
            x.iloc[i], x.iloc[j] = x.iloc[j], x.iloc[i]

    @staticmethod
    def split(x, k):
        return x[:int(len(x)*k)].copy(), x[int(len(x)*k):].copy()

    @staticmethod
    def normalization(x):
        values = x.iloc[:,:-1]
        columnNames = values.columns.tolist()
        for column in columnNames:
            data = values.loc[:, column]
            min1 = min(data)
            max1 = max(data)
            for row in range(0, len(data), 1):
                x.loc[row, column] = (x.loc[row, column]-min1)/(max1-min1)
                
    @staticmethod
    def stan_method(x):
        for column in x.columns[:-1]:
            x[column] = (x[column] - x[column].mean()) / x[column].std()
    
    @staticmethod
    def norm_method(x):
        for column in x.columns[:-1]:
            x[column] = (x[column] - x[column].min()) / (x[column].max() - x[column].min())
        return x

In [ ]:
#To jest nasza modyfikacja: minkowski + voting softsetem
class KNN:
    @staticmethod
    # metryka Minkowskiego dla m=1,2,3,4 i zwracająca max z nich
    def minkowski_distance(x, y):
        distances_temp = []
        for i in range(1, 5):
            distance = 0
            for j in range(len(x)):
                distance += math.pow(abs(x[j] - y[j]), i)
            distances_temp.append(math.pow(distance, 1/i))
        return max(distances_temp)

    @staticmethod
    def cluster(sample, training, k):
        distances = []
        temporary = training.iloc[:,:-2]
        for i in range(len(temporary)):
            distances.append(KNN.minkowski_distance(temporary.iloc[i], sample[:-1]))
        training["distance"] = distances
        training = training.sort_values('distance', ascending=True)[:k]

        gatunki = [0,1]
        votes = {
            0: 0,
            1: 0,
        }

        # dla każdej klasy (są 3)
        for i in range(len(gatunki)):
            # licznik i mianownik ale po angielsku
            numerator = denominator = 0
            # dla każdego elementu ze zbioru wyznaczonych sąsiadów
            for j in range(k):
                # jeżeli sąsiad należy do obecnie przetwarzanej klasy:
                if training.iloc[j]["output"] == gatunki[i]:
                    # sprawdzenie, czy dystans jest większy od zera
                    if training.iloc[j]["distance"] > 0:
                        numerator += (1 /
                                      math.pow(training.iloc[j]["distance"], 2))
                    else:
                        numerator += 1  # w przypadku dystansu równego 0, dodajemy wartość 1 do licznika
                # dorzucamy do mianownika dystans
                # sprawdzenie, czy dystans jest większy od zera
                if training.iloc[j]["distance"] > 0:
                    denominator += (1 /
                                    math.pow(training.iloc[j]["distance"], 2))
                else:
                    denominator += 1  # w przypadku dystansu równego 0, dodajemy wartość 1 do mianownika
            # dla klasy dodajemy otrzymany wynik
            if denominator != 0:
                votes[gatunki[i]] += numerator / denominator
        return max(votes, key=votes.get)

    @staticmethod
    def calculate_accuracy(training, validation, k):
        correct = 0
        results = []
        for i in range(len(validation)):
            result = KNN.cluster(validation.iloc[i], training, k)
            results.append(result)
            if validation.iloc[i]["output"] == result:
                correct += 1
        return ((correct / len(validation)) * 100, results)

In [ ]:
#Klasyczny KNN: euklides, voting ilościowy
class KNN3:
    @staticmethod
    def knn(sample, train, num):
        distance = []
        for i in range(len(train)):
            temp = 0
            for j in range(len(sample)-1):
                temp += math.pow(sample[j]-train.iloc[i][j],2)
            distance.append(math.sqrt(temp))
        train["distance"] = distance
        
        #sortowanie po distance
        train = train.sort_values("distance")
        #print(train)
        
        #głosowanie
        votes = {}
        for i in range(num):
            if (votes.get(train.iloc[i]["output"]) is None):
                votes[train.iloc[i]["output"]] = 1
            else:
                votes[train.iloc[i]["output"]] += 1
        
        return max(votes, key=lambda x: votes[x])
    
    @staticmethod
    def accuracy(train, test, num):
        poprawne = 0
        for i in range(len(test)):
            result = KNN2.knn(test.iloc[i,:-1], train, num)
            if result == test.iloc[i]["output"]:
                poprawne += 1
        return (poprawne/len(test))*100

In [ ]:
###TESTY###

In [ ]:
#od nowa
tablicaKNN2 = [] #zmodyfikowany KNN bez obcięć
tablicaKNN3 = [] #klasyczny KNN bez obcięć

conf_mat2 = []#zmodyfikowany KNN bez obcięć
for i in range(50):
    tabKNN2 = [] #zmodyfikowany KNN bez obcięć
    tabKNN3 = [] #klasyczny KNN bez obcięć
    
    DataProcessing.shuffle(data)
    DataProcessing.norm_method(data)
    data_train, data_test = DataProcessing.split(data, 0.7)
    
    max_valKNN2 = 1
    max_valKNN3 = 1
    
    idxKNN2 = 0
    idxKNN3 = 0
    
    tmp_conf2 = list()
    
    for i in range(1,11):
        tmpKNN2, result2 = KNN.calculate_accuracy(data_train.iloc[:,:14], data_test.iloc[:,:14],i)
        tmpKNN3 = KNN3.accuracy(data_train.iloc[:,:14], data_test.iloc[:,:14],i) #tu zaś kurwa moze byc to ze ta kolumna distance wchodzi i kolejny ja bierze
        
        tabKNN2.append(tmpKNN2)
        tabKNN3.append(tmpKNN3)
        
        if tmpKNN2 > max_valKNN2:
            max_valKNN2 = tmpKNN2
            idxKNN2 = i
            tmp_conf2 = result2
        if tmpKNN3 > max_valKNN3:
            max_valKNN3 = tmpKNN3
            idxKNN3 = i

    tablicaKNN2.append(tabKNN2)
    tablicaKNN3.append(tabKNN3)
    
    conf_mat1.append(tmp_conf1)
    conf_mat2.append(tmp_conf2)
    print(" KNN2: ",max_valKNN2,idxKNN2," KNN3: ",max_valKNN3,idxKNN3)

In [ ]:
print(np.mean(tablicaKNN1, axis=0))
print(np.mean(tablicaKNN2, axis=0))
print(np.mean(tablicaKNN3, axis=0))
a = (np.mean(tablicaKNN1, axis=0))
b = (np.mean(tablicaKNN2, axis=0))
c = (np.mean(tablicaKNN3, axis=0))
print(np.mean(a))
print(np.mean(b))
print(np.mean(c))

In [ ]:
import matplotlib.pyplot as plt

k = [1,2,3,4,5,6,7,8,9,10]
df1 = pd.DataFrame({"k":k, "Accuracy":b.tolist()})
df2 = pd.DataFrame({"k":k, "Accuracy":c.tolist()})
sns.lineplot(x = "k", y= "Accuracy",data=df1, color="#e6194b", label="Our modification", linewidth=2)
sns.lineplot(x = "k", y= "Accuracy",data=df2, color="#0082c8", label="KNN", linewidth=2)
plt.xticks(k)
plt.legend()
plt.grid(True, linewidth=0.1, color="black", linestyle="--")
plt.show()

In [ ]:
#confusion matrix
from sklearn.metrics import confusion_matrix

cf_matrix = confusion_matrix(data_test["output"].tolist(), tmp_conf2)
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt='.2%', cmap='Blues')

In [ ]:
#recall
from sklearn.metrics import recall_score
tablicaKNN = []
recallKNN = []

conf_mat = []
for i in range(50):
    tabKNN = []
    recKNN = []
    
    DataProcessing.shuffle(data)
    DataProcessing.norm_method(data)
    data_train, data_test = DataProcessing.split(data, 0.7)
    
    max_valKNN = 1
    idxKNN = 0
    
    tmp_conf = list()
    tmp_rec = 0
    for i in range(1,100):
        tmpKNN, result = KNN.calculate_accuracy(data_train.iloc[:,:14], data_test.iloc[:,:14],i)
        rec = recall_score(data_test["output"].tolist(), result)
        
        tabKNN.append(tmpKNN)
        recKNN.append(rec)
        
        if tmpKNN > max_valKNN:
            max_valKNN = tmpKNN
            idxKNN = i
            tmp_conf = result
        if rec > tmp_rec:
            tmp_rec = rec
            
    tablicaKNN.append(tabKNN)
    recallKNN.append(recKNN)
    conf_mat.append(tmp_conf)
    
    print("KNN:", max_valKNN,"  ", idxKNN,"  ",rec)

In [ ]:
recallplot = np.mean(recallKNN,axis=0)

In [ ]:
import matplotlib.pyplot as plt

k = list(range(1,100))
df1 = pd.DataFrame({"k":k, "Recall":recallplot.tolist()})
sns.lineplot(x = "k", y= "Recall",data=df1, color="#e6194b", label="Our modification", linewidth=2)
#plt.xticks(k)
plt.legend()
plt.grid(True, linewidth=0.1, color="black", linestyle="--")
plt.show()